In [9]:
import pandas as pd
import numpy as np
import itertools
import sys
from geopy.distance import geodesic
from tqdm import tqdm
import random

In [10]:
cities = pd.read_csv('cities/russia.csv', header=None, names=['City','x', 'y'])
cities

,City,x,y
0,Abakan,53.72,91.4300
1,Achinsk,56.28,90.5000
2,Almetyevsk,54.90,52.3100
3,Angarsk,52.57,103.9100
4,Arkhangelsk,64.57,40.5300
...,...,...,...
162,Yuzhno‐Sakhalinsk,46.95,142.7400
163,Zelenograd,55.94,37.2900
164,Zheleznodorozhnyy,55.75,38.0167
165,Zhukovskiy,55.55,38.2500


Section to calculate the distances of each city

In [ ]:
dist_df = np.zeros((cities.shape[0], cities.shape[0]))
for c1, c2 in itertools.combinations(cities.itertuples(), 2):
    distance = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
    dist_df[c1.Index, c2.Index] = distance
    dist_df[c2.Index, c1.Index] = distance
dist_df

City,Abakan,Achinsk,Almetyevsk,Angarsk,Arkhangelsk,Armavir,Artyom,Arzamas,Astrakhan,Balakovo,...,Yakutsk,Yaroslavl,Yekaterinburg,Yelets,Yoshkar‐Ola,Yuzhno‐Sakhalinsk,Zelenograd,Zheleznodorozhnyy,Zhukovskiy,Zlatoust
City,,,,,,,,,,,,,,,,,,,,,
Abakan,0.000000,291.128718,2516.334423,843.679166,3050.332757,3697.257752,3169.542974,3025.825566,3160.248068,2898.509715,...,2409.701565,3193.299672,1972.112907,3459.139535,2746.566102,3645.954058,3398.351696,3362.128417,3355.471138,2050.063925
Achinsk,291.128718,0.000000,2381.820027,961.257590,2816.838329,3620.898730,3260.367862,2870.570076,3100.029756,2784.049159,...,2294.408961,3012.590522,1824.669088,3316.461657,2585.782447,3666.201243,3227.124159,3193.609126,3188.932911,1926.108290
Almetyevsk,2516.334423,2381.820027,0.000000,3337.280762,1260.232903,1360.309035,5638.256356,543.704000,997.260501,438.619460,...,4309.128156,826.805764,562.033263,943.898781,339.284923,5966.351813,956.216476,910.555566,896.090539,470.051232
Angarsk,843.679166,961.257590,3337.280762,0.000000,3734.879772,4540.483966,2325.976767,3831.371265,4003.865093,3730.900252,...,1859.595679,3971.640225,2783.175743,4275.243329,3546.938901,2830.126952,4187.842854,4154.620046,4150.081920,2876.386344
Arkhangelsk,3050.332757,2816.838329,1260.232903,3734.879772,0.000000,2179.873402,5835.855292,1039.865081,2080.005815,1457.720557,...,4110.679351,775.245358,1382.871147,1338.287494,969.273889,5942.536632,977.787883,992.334111,1012.763485,1486.778952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yuzhno‐Sakhalinsk,3645.954058,3666.201243,5966.351813,2830.126952,5942.536632,7270.784707,920.222105,6383.496878,6762.725813,6394.279094,...,1870.295310,6425.583976,5407.394211,6848.641810,6096.221230,0.000000,6666.029764,6644.945432,6648.021689,5543.640852
Zelenograd,3398.351696,3227.124159,956.216476,4187.842854,977.787883,1247.540201,6436.554491,415.534566,1302.593214,812.919780,...,4899.218964,244.653540,1436.056718,380.160639,659.112548,6666.029764,0.000000,50.197009,74.298383,1407.513053
Zheleznodorozhnyy,3362.128417,3193.609126,910.555566,4154.620046,992.334111,1216.767020,6408.346845,368.339882,1257.604584,762.814741,...,4883.981078,237.172893,1397.027870,352.094864,619.090180,6644.945432,50.197009,0.000000,26.674789,1364.472355


In [12]:
# Greedy heuristic to generate initial population
def greedy_heuristic(dists, population_size):
    def nearest_neighbor(start):
        n = len(dists)
        visited = [False] * n
        path = [start]
        visited[start] = True
        for _ in range(n - 1):
            last = path[-1]
            nearest = None
            nearest_dist = float('inf')
            for i in range(n):
                if not visited[i] and dists[last][i] < nearest_dist:
                    nearest = i
                    nearest_dist = dists[last][i]
            path.append(nearest)
            visited[nearest] = True
        path.append(start)
        return path

    population = []
    for _ in range(population_size // 2):
        start = random.randint(0, len(dists) - 1)
        path = nearest_neighbor(start)
        population.append(path)

    # Add random paths to increase diversity
    for _ in range(population_size // 2):
        path = list(range(1, len(dists)))
        random.shuffle(path)
        path = [0] + path + [0]
        population.append(path)

    return population

# Evolutionary Algorithm with Inver-Over Crossover for TSP
def evolutionary_algorithm(dists, population_size, mutation_rate, generations):
    def calculate_cost(path):
        return sum(dists[path[i], path[i + 1]] for i in range(len(path) - 1))

    def inver_over_crossover(parent1, parent2):
        child = parent1[:]
        for i in range(1, len(child) - 1):
            if random.random() < 0.5:
                j = random.choice(range(1, len(child) - 1))
                if random.random() < 0.5:
                    child[i:j+1] = reversed(child[i:j+1])
                else:
                    child[i], child[j] = child[j], child[i]
        return child

    def mutate(path):
        if random.random() < mutation_rate:
            i, j = random.sample(range(1, len(path) - 1), 2)
            path[i], path[j] = path[j], path[i]

    def select_parents(population):
        return random.choices(population, weights=[1 / calculate_cost(p) for p in population], k=2)

    population = greedy_heuristic(dists, population_size)
    best_path = min(population, key=calculate_cost)
    best_cost = calculate_cost(best_path)

    for generation in tqdm(range(generations), desc="Evolutionary Algorithm"):
        new_population = []
        for _ in range(population_size):
            parent1, parent2 = select_parents(population)
            child = inver_over_crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)
        population = new_population

        current_best_path = min(population, key=calculate_cost)
        current_best_cost = calculate_cost(current_best_path)
        if current_best_cost < best_cost:
            best_path = current_best_path
            best_cost = current_best_cost

    return best_cost, best_path

# Example usage
if __name__ == '__main__':
    population_size = 100
    mutation_rate = 0.01
    generations = 1000

    # Solve the TSP using the Evolutionary Algorithm with Inver-Over Crossover
    opt_cost, opt_path = evolutionary_algorithm(dist_df, population_size, mutation_rate, generations)

    # Convert the path indices to city names
    opt_path_cities = [cities['City'][i] for i in opt_path]

    print(f"Approximate cost: {opt_cost:.2f} km")
    print(f"Approximate path: {opt_path_cities}")

KeyError: 109